<a href="https://colab.research.google.com/github/wprzem/ball_tracker/blob/main/colab_train_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf ball_tracker
!git clone https://github.com/wprzem/ball_tracker.git

Cloning into 'ball_tracker'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 1684 (delta 3), reused 7 (delta 2), pack-reused 1675
Receiving objects: 100% (1684/1684), 583.22 MiB | 35.83 MiB/s, done.
Resolving deltas: 100% (396/396), done.
Checking out files: 100% (691/691), done.


In [2]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14730, done.
remote: Total 14730 (delta 0), reused 0 (delta 0), pack-reused 14730
Receiving objects: 100% (14730/14730), 13.25 MiB | 5.64 MiB/s, done.
Resolving deltas: 100% (10018/10018), done.


In [3]:
!ls darknet/

3rdparty	DarknetConfig.cmake.in	include		       results
build		darknet_images.py	json_mjpeg_streams.sh  scripts
build.ps1	darknet.py		LICENSE		       src
build.sh	darknet_video.py	Makefile	       vcpkg.json
cfg		data			net_cam_v3.sh	       video_yolov3.sh
cmake		image_yolov3.sh		net_cam_v4.sh	       video_yolov4.sh
CMakeLists.txt	image_yolov4.sh		README.md


In [4]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

In [5]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!cat cfg/yolov3_training.cfg

[net]
# Testing
batch=1
subdivisions=1
# Training
# batch=64
# subdivisions=16
width=416
height=416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 500200
policy=steps
steps=400000,450000
scales=.1,.1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

# Downsample

[convolutional]
batch_normalize=1
filters=64
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=32
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[shortcut]
from=-3
activation=linear

# Downsample

[convolutional]
batch_normalize=1
filters=128
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[shortcut]
from=-3
activation=linear

[convolutiona

In [6]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=64/' cfg/yolov3_training.cfg
!sed -i 's/max_batches=500200/max_batches=4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!ls /content/drive/MyDrive
!mkdir /content/drive/MyDrive/yolov3

 backup			      'Finanse VC.gsheet'
 basin_entropy.ipynb	       return_182006.pdf
'Colab Notebooks'	       yolov3
 control_staly1_2mil_d01.txt   yolov3_training_last.weights
 darknet.zip		       yolov3_v2
mkdir: cannot create directory ‘/content/drive/MyDrive/yolov3’: File exists


In [14]:
!echo "ball" > /content/darknet/data/obj.names
!echo -e "classes= 1\ntrain = /content/darknet/data/train.txt\nvalid = /content/darknet/data/test.txt\nnames = /content/darknet/data/obj.names\nbackup = /content/drive/MyDrive" > /content/darknet/data/obj.data
!cat /content/darknet/data/obj.data
!mkdir /content/darknet/data/obj
!mkdir /content/darknet/data/obj_test

classes= 1
train = /content/darknet/data/train.txt
valid = /content/darknet/data/test.txt
names = /content/darknet/data/obj.names
backup = /content/drive/MyDrive
mkdir: cannot create directory ‘/content/darknet/data/obj’: File exists


In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-03-08 20:22:44--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  42.4MB/s    in 4.0s    

2021-03-08 20:22:49 (38.5 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [12]:
%cp /content/ball_tracker/train_images_renamed/*.jpg /content/darknet/data/obj/
%cp /content/ball_tracker/train_images_renamed/*.jpeg /content/darknet/data/obj/
%cp /content/ball_tracker/train_images_renamed/*.txt /content/darknet/data/obj/

cp: cannot stat '/content/ball_tracker/train_images_renamed/*.jpeg': No such file or directory


In [19]:
import glob
%cd /content/darknet/
images_list_jpg = glob.glob("data/obj/*.jpg")
images_list_png = glob.glob("data/obj/*.png")
images_list_jpg +=images_list_png
print(images_list_jpg)

/content/darknet
['data/obj/112.jpg', 'data/obj/259.jpg', 'data/obj/14.jpg', 'data/obj/116.jpg', 'data/obj/104.jpg', 'data/obj/46.jpg', 'data/obj/278.jpg', 'data/obj/217.jpg', 'data/obj/170.jpg', 'data/obj/19.jpg', 'data/obj/198.jpg', 'data/obj/151.jpg', 'data/obj/113.jpg', 'data/obj/315.jpg', 'data/obj/221.jpg', 'data/obj/167.jpg', 'data/obj/95.jpg', 'data/obj/11.jpg', 'data/obj/326.jpg', 'data/obj/137.jpg', 'data/obj/196.jpg', 'data/obj/304.jpg', 'data/obj/36.jpg', 'data/obj/100.jpg', 'data/obj/136.jpg', 'data/obj/144.jpg', 'data/obj/247.jpg', 'data/obj/152.jpg', 'data/obj/271.jpg', 'data/obj/148.jpg', 'data/obj/311.jpg', 'data/obj/273.jpg', 'data/obj/220.jpg', 'data/obj/134.jpg', 'data/obj/39.jpg', 'data/obj/63.jpg', 'data/obj/60.jpg', 'data/obj/184.jpg', 'data/obj/264.jpg', 'data/obj/4.jpg', 'data/obj/139.jpg', 'data/obj/240.jpg', 'data/obj/261.jpg', 'data/obj/98.jpg', 'data/obj/234.jpg', 'data/obj/75.jpg', 'data/obj/115.jpg', 'data/obj/81.jpg', 'data/obj/308.jpg', 'data/obj/329.jp

In [20]:
%cd /content/darknet/
%ls data
file = open("/content/darknet/data/train.txt",'w')
file.write("\n".join(images_list_jpg))
file.close()

/content/darknet
9k.tree     eagle.jpg    imagenet.labels.list      obj.data          person.jpg
coco9k.map  giraffe.jpg  imagenet.shortnames.list  obj.names         scream.jpg
coco.names  goal.txt     labels/                   obj_test/         test.txt
dog.jpg     horses.jpg   obj/                      openimages.names  voc.names


In [17]:
!ls /content/ball_tracker/
%cp /content/drive/MyDrive/test_images/*.jpg /content/darknet/data/obj_test/
%cp /content/drive/MyDrive/test_images/*.jpeg /content/darknet/data/obj_test/
%cp /content/drive/MyDrive/test_images/*.txt /content/darknet/data/obj_test/

CMakeLists.txt		images	    README.md  train_images_renamed
colab_train_yolo.ipynb	prototypes  src


In [21]:
%cd /content/darknet/
images_list_jpg = glob.glob("data/obj_test/*.jpg")
print(images_list_jpg)

/content/darknet
['data/obj_test/images5.jpg', 'data/obj_test/9deda680298d2aebcc48e4be7e062a2d.jpg', 'data/obj_test/images50.jpg', 'data/obj_test/images2.jpg', 'data/obj_test/RUB_4130.jpg', 'data/obj_test/AR1V0066-696x464.jpg', 'data/obj_test/1600x900_1568280276272.GettyImages_1042577556.jpg', 'data/obj_test/images47.jpg', 'data/obj_test/rpkosf9bneu59pyz9dytvgixhxwiiq92.jpg', 'data/obj_test/mondiali-pallavolo-2018-italia-finlandia.jpg', 'data/obj_test/images65.jpg', 'data/obj_test/images14.jpg', 'data/obj_test/images57.jpg', 'data/obj_test/IMG_2307-696x464.jpg', 'data/obj_test/images36.jpg', 'data/obj_test/1200px-Mondiali_di_pallavolo_2014_281543712722029.jpg', 'data/obj_test/images40.jpg', 'data/obj_test/images66.jpg', 'data/obj_test/Monza_Vibo-696x365.jpg', 'data/obj_test/McDonnell.jpg', 'data/obj_test/Mlbjerseys-Volleyball.jpg', 'data/obj_test/fondamentali_pallavolo.jpg', 'data/obj_test/TN-PC-9-696x499.jpg', 'data/obj_test/images4.jpg', 'data/obj_test/MG_5655-low-696x464.jpg', 'data

In [22]:
%cd /content/darknet/
%ls data
file = open("/content/darknet/data/test.txt",'w')
file.write("\n".join(images_list_jpg))
file.close()

/content/darknet
9k.tree     giraffe.jpg               labels/    openimages.names  voc.names
coco9k.map  goal.txt                  obj/       person.jpg
coco.names  horses.jpg                obj.data   scream.jpg
dog.jpg     imagenet.labels.list      obj.names  test.txt
eagle.jpg   imagenet.shortnames.list  obj_test/  train.txt


In [23]:
%ls /content/drive/MyDrive/


 backup/                       return_182006.pdf
 basin_entropy.ipynb           test_images/
'Colab Notebooks'/             yolov3/
 control_staly1_2mil_d01.txt   yolov3_training_last.weights
 darknet.zip                   yolov3_v2/
'Finanse VC.gsheet'


In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg /content/drive/MyDrive/yolov3_training_last.weights -dont_show

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
 total_bbox = 7254, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.753080), count: 1, class_loss = 0.000106, iou_loss = 0.206233, total_loss = 0.206339 
 total_bbox = 7255, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.839798), count: 1, class_loss = 0.000005, iou_loss = 0.045475, total_loss = 0.0454